In [2]:
import pypsa 
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
n0 = pypsa.Network('../networks/elec_grid-2040_11-supply_redz.nc')

INFO:pypsa.io:Imported network elec_grid-2040_11-supply_redz.nc has buses, carriers, generators, lines, loads, storage_units


In [12]:
n0.optimize(solver_name="xpress", solver_options={"lpflags":4,"crossover":0})

INFO:linopy.model: Solve linear problem using Xpress solver
INFO:linopy.model:Solver options:
 - lpflags: 4
 - crossover: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 9/9 [00:01<00:00,  6.87it/s]
INFO:linopy.io: Writing time: 11.14s


Using the Community license in this session. If you have a full Xpress license, pass the full path to your license file to xpress.init(). If you want to use the FICO Community license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('/home/pklein/anaconda3/envs/pypsa-earth-updated/lib/python3.10/site-packages/xpress/license/community-xpauth.xpr')
 
Reading Problem linopy-problem-koqi39tw
Problem Statistics
     4301233 (      0 spare) rows
     1787147 (      0 spare) structural columns
     8697074 (      0 spare) non-zero elements
MIP Entity Statistics
           0 entities        0 sets        0 set members
FICO Xpress v9.1.2, Hyper, solve started 15:29:39, Oct 27, 2023
Heap usage: 1261MB (peak 2016MB, 51MB system)
Minimizing LP linopy-problem-koqi39tw using up to 20 threads and up to 125GB memory, with these control settings:
OUTPUTLOG = 1
CROSSOVER = 0
LPFLAGS = 4
Original problem has:
   4301233 rows      1787147 cols   

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 1787147 primals, 4301233 duals
Objective: 2.09e+11
Solver model: available
Solver message: lp_optimal

/home/pklein/.local/lib/python3.10/site-packages/linopy/common.py:101: UserWarning:

Coordinates across variables not equal. Perform outer join.

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Kirchhoff-Voltage-Law were not assigned to the network.


('ok', 'optimal')

In [16]:
n0.generators.p_nom_opt.groupby(n0.generators.carrier).sum()

carrier
ccgt-gas         6923.425552
coal            22927.000011
hydro             683.020000
hydro_import     1764.000413
nuclear          1854.000009
ocgt-diesel      1593.214914
ocgt-gas          425.000000
onwind          35653.651409
solar_pv        45269.173851
Name: p_nom_opt, dtype: float64